# JACOBIAN

## Overview
The **Jacobian matrix** is a fundamental concept in multivariable calculus and mathematical optimization. It generalizes the derivative of a function to higher dimensions. For a vector-valued function $\mathbf{f}: \mathbb{R}^n \to \mathbb{R}^m$, the Jacobian matrix $J$ is an $m \times n$ matrix where each entry is the partial derivative of one component of $\mathbf{f}$ with respect to one variable:

```math
J = \left[ \frac{\partial f_i}{\partial x_j} \right], \quad J_{ij} = \frac{\partial f_i}{\partial x_j}
```

Each row corresponds to the gradient of one output function with respect to all input variables. The Jacobian describes how small changes in the input variables affect the outputs. It is widely used in:
- Sensitivity analysis
- Nonlinear optimization
- Solving systems of nonlinear equations
- Machine learning (e.g., backpropagation)
- Robotics and control systems

See the [CasADi documentation](https://web.casadi.org/docs/) for more details on the [Jacobian](https://github.com/casadi/casadi/wiki/L_1cv) and its applications.

This example function is provided as-is without any representation or warranty of accuracy.

## Usage
To use the function in Excel:

```excel
=JACOBIAN(expression, var_values)
```
- `expression` (string, required): The mathematical expression as a string. Example: `"x**2 * y"`
- `var_values` (list[list], required): A 2D list where the first row contains variable names and the second row contains their values. Example: `{"x","y";2.0,3.0}`

The function returns a 2D list of floats representing the Jacobian matrix, or a string error message if calculation fails.

## Examples
```excel
=JACOBIAN("x**2 * y", {"x","y";2.0,3.0})
```
Expected output:

| ∂f/∂x | ∂f/∂y |
|-------|-------|
|  12   |   4   |

```excel
=JACOBIAN("ca.sin(x) + y**3", {"x","y";1.0,2.0})
```
Expected output:

| ∂f/∂x  | ∂f/∂y |
|--------|-------|
| 0.5403 |  12   |

## CasADi Function Names
When writing expressions, you may use standard function names (e.g., `sin(x)`, `exp(x)`) or CasADi equivalents (e.g., `ca.sin(x)`, `ca.exp(x)`). Standard names will be automatically converted to CasADi equivalents. See table below:

| Standard Name | CasADi Equivalent |
|--------------|-------------------|
| sin(x)       | ca.sin(x)         |
| cos(x)       | ca.cos(x)         |
| tan(x)       | ca.tan(x)         |
| exp(x)       | ca.exp(x)         |
| log(x)       | ca.log(x)         |
| sqrt(x)      | ca.sqrt(x)        |
| atan(x)      | ca.atan(x)        |
| asin(x)      | ca.asin(x)        |
| acos(x)      | ca.acos(x)        |
| sinh(x)      | ca.sinh(x)        |
| cosh(x)      | ca.cosh(x)        |
| tanh(x)      | ca.tanh(x)        |
| fabs(x)      | ca.fabs(x)        |
| floor(x)     | ca.floor(x)       |
| ceil(x)      | ca.ceil(x)        |

If you use a standard function name, it will be automatically converted to the CasADi equivalent.

In [ ]:
import casadi as ca
import re

def jacobian(expression, var_values):
    """
    Calculates the Jacobian matrix of a mathematical expression.

    Args:
        expression (str): The mathematical expression as a string (e.g., "x**2 + y").
        var_values (list[list]): A 2D list, where the first inner list contains the names of the variables
            (e.g., ['x', 'y']) and the second inner list contains their corresponding values
            at which to evaluate the Jacobian (e.g., [1.0, 2.0]).

    Returns:
        list[list[float]]: On success, a 2D list of floats representing the Jacobian matrix.
        str: Error message if calculation fails.

    This example function is provided as-is without any representation or warranty of accuracy.
    """
    # Mapping of standard function names to CasADi equivalents
    casadi_func_map = {
        r'(?<![\w.])sin\s*\(': 'ca.sin(',
        r'(?<![\w.])cos\s*\(': 'ca.cos(',
        r'(?<![\w.])tan\s*\(': 'ca.tan(',
        r'(?<![\w.])exp\s*\(': 'ca.exp(',
        r'(?<![\w.])log\s*\(': 'ca.log(',
        r'(?<![\w.])sqrt\s*\(': 'ca.sqrt(',
        r'(?<![\w.])atan\s*\(': 'ca.atan(',
        r'(?<![\w.])asin\s*\(': 'ca.asin(',
        r'(?<![\w.])acos\s*\(': 'ca.acos(',
        r'(?<![\w.])sinh\s*\(': 'ca.sinh(',
        r'(?<![\w.])cosh\s*\(': 'ca.cosh(',
        r'(?<![\w.])tanh\s*\(': 'ca.tanh(',
        r'(?<![\w.])fabs\s*\(': 'ca.fabs(',
        r'(?<![\w.])floor\s*\(': 'ca.floor(',
        r'(?<![\w.])ceil\s*\(': 'ca.ceil(',
    }
    try:
        if not isinstance(expression, str):
            return "expression must be a string."
        if not isinstance(var_values, list) or len(var_values) != 2:
            return "var_values should be a list of two lists: [[variable_names], [values]]"
        var_names = var_values[0]
        var_point_values = var_values[1]
        if not all(isinstance(name, str) for name in var_names):
            return "Variable names must be strings."
        if not all(isinstance(val, (int, float)) for val in var_point_values):
            return "Variable values must be numbers."
        if len(var_names) != len(var_point_values):
            return "Mismatch in the number of variable names and their values."
        # Automatically convert standard function names to CasADi equivalents
        expr = expression
        for pat, repl in casadi_func_map.items():
            expr = re.sub(pat, repl, expr)
        # Create symbolic variables
        sym_vars = [ca.MX.sym(name) for name in var_names]
        vars_dict = {name: sym_vars[i] for i, name in enumerate(var_names)}
        try:
            eval_locals = vars_dict.copy()
            eval_locals['ca'] = ca
            casadi_expr = eval(expr, {**globals(), **locals()}, eval_locals)
        except NameError as e:
            undefined_var = str(e).split("'")[1]
            if undefined_var not in var_names:
                return f"Mismatch between variables in expression and provided values. Undefined variable: {undefined_var}"
            return f"Invalid expression string. Error: {e}"
        except Exception as e:
            return f"Invalid expression string. Error: {e}"
        J = ca.jacobian(casadi_expr, ca.vertcat(*sym_vars))
        jacobian_func = ca.Function('jacobian_func', sym_vars, [J])
        result_matrix = jacobian_func(*var_point_values)
        if isinstance(result_matrix, ca.DM):
            return result_matrix.full().tolist()
        else:
            return "Error during CasADi calculation: Unexpected result type."
    except Exception as e:
        return str(e)

In [ ]:
%pip install -q ipytest
import ipytest
ipytest.autoconfig()

demo_cases = [
    ["x**2 * y", [["x", "y"], [2.0, 3.0]]],
    ["sin(x) + y**3", [["x", "y"], [1.0, 2.0]]],
    ["exp(x) + log(y)", [["x", "y"], [1.0, 2.0]]],
]

def is_valid_type(val):
    if isinstance(val, (float, bool, str)):
        return True
    if isinstance(val, list):
        return all(isinstance(row, list) and all(isinstance(x, (float, bool, str)) for x in row) for row in val)
    return False

import pytest
@pytest.mark.parametrize("expression, var_values", demo_cases)
def test_demo_cases(expression, var_values):
    result = jacobian(expression, var_values)
    print(f"test_demo_cases output for {expression}: {result}")
    assert is_valid_type(result), f"Output type is not valid. Got: {type(result)} Value: {result}"

def test_invalid_expression():
    result = jacobian("x**2 * z", [["x", "y"], [2.0, 3.0]])
    print(f"test_invalid_expression output: {result}")
    assert isinstance(result, str) and ("Undefined variable" in result or "Invalid expression" in result)

def test_invalid_var_values():
    result = jacobian("x**2 * y", [["x", "y"], [2.0]])
    print(f"test_invalid_var_values output: {result}")
    assert isinstance(result, str) and "Mismatch" in result

ipytest.run('-s')

In [ ]:
# Interactive Demo
import gradio as gr

demo = gr.Interface(
    fn=jacobian,
    inputs=[
        gr.Textbox(label="Expression", value=demo_cases[0][0]),
        gr.DataFrame(headers=["", ""], label="Variable Names and Values", row_count=2, col_count=len(demo_cases[0][1][0]), type="array", value=demo_cases[0][1])
    ],
    outputs=gr.DataFrame(headers=["∂f/∂x", "∂f/∂y"], label="Jacobian Matrix"),
    examples=demo_cases,
    description="Calculate the Jacobian matrix of a mathematical expression with respect to specified variables. This demo is provided as-is without any representation of accuracy.",
    flagging_mode="never",
    fill_width=True,
)
demo.launch()